## <h1><center>Análisis de relaciones entre variables y selección de predictores significativos en el resultado farmacocinético al séptimo día.</center></h1> 

Cuaderno de Python enmarcado en el Trabajo de Fin de Grado: "Diseño de regímenes de tratamiento óptimos en pacientes hipertensos con insuficiencia renal. Un enfoque desde el Aprendizaje por Refuerzo". Este cuaderno contiene el análisis descriptivo del dataset sintético de 10.000 pacientes virtuales tratados durante siete días con 10 mg del fármaco antitensivo Benazepril, inhibidor de la enzima convertidora de la angiotensina (ECA), generado por un simulador poblacional de farmacocinética basada en la fisiología (PopPBPK) desarrollado por un grupo de investigación en el Centro de investigación Pharmaceutical Engineering GmbH en Graz, Austria.

Se realiza un estudio estadístico de la relación entre las variables antropométricas y fisiológicas de los pacientes y el resultado de las variables farmacocinéticas ABC (Área Bajo la Curva de cononcentración plasmática en el tiempo) y Cmáx (Concentración plasmática máxima) de metabolito al séptimo día, para encontrar variables significativas en la predicción del resultado farmacocinético. Adicionalmente, se realiza la selección "consciente" o "guiada", de acuerdo con el dominio del problema y los objetivos de implementación del proyecto de investigación final, de predictores significativos para la identificación de grupos de estratificación de la población y para la construcción del modelo computacional de aprendizaje por refuerzo para el diseño de regímenes de tratamiento dinámicos optimizados para cada subgrupo de estratificación.

Finalmente, se practica un estudio de variabilidad intragrupo para la evaluación de los grupos definidos mediante la estratificación de la población de pacientes,


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dfply import *
from plotnine import *
import seaborn as sns
import scipy.stats as stats
from typing import List, Dict
import statsmodels.api as sm
from scipy.stats import norm, kstest
import pingouin as pg

CARGA DE LOS DATOS

In [3]:
# Carga de los datos de 10.000 pacientes con diferentes estadíos de insuficiencia renal tratados con una dosis de 10mg de Benazepril al día durante siete días.

data = pd.read_excel('A2_DOK_10000patStrat_10mg_V01.xlsx', sheet_name = '10mg')

FUNCIONES

In [ ]:
# Discretización de la variable de BMI (5):
# Delgadez severa: < 16, Delgadez media-moderada: 16-18.5, Normal: 18.5 – 25, Sobrepeso: 25-30, Obeso I: > 30

def BMI_subgroups(BMI):
    if (BMI < 16):
        subgroup = "Severe thinness"
    elif (16 <= BMI < 18.5):
        subgroup = "Mild-moderate thinness"
    elif (18.5 <= BMI < 25):
        subgroup = "Normal"
    elif (25 <= BMI < 30):
        subgroup = "Overweight"
    elif (BMI >= 30):
        subgroup = "Obese"
    return subgroup


In [69]:
# Categorización de la variable cuantitativa Cmax en función de la disposición de fármaco (3):
# Sub-disposición: < 131 ng/ml, Disposición adecuada: 131-219 ng ml, Sobre-disposición: > 219 ng/ml

def Cmax_subgroups(Cmax):
    if (Cmax < 131):
        subgroup = "Sub-disposición"
    elif (131 <= Cmax <= 219):
        subgroup = "Disposición adecuada"
    elif (Cmax > 219):
        subgroup = "Sobre-disposición"
    return subgroup

In [70]:
# Categorización de la variable cuantitativa ABC en función de la disposición de fármaco (3):
# Sub-disposición: < 470 ng/ml·h, Disposición adecuada: 474-1210 ng/ml·h, Sobre-disposición: >  1210 ng/ml·h

def AUC_subgroups(AUC):
    if (AUC < 470):
        subgroup = "Sub-disposición"
    elif (470 <= AUC <= 1210):
        subgroup = "Disposición adecuada"
    elif (AUC > 1210):
        subgroup = "Sobre-disposición"
    return subgroup

In [43]:
# color de celdas de un dataset para el estudio de normalidad

def _color_red_or_green(val):   
    color = 'red' if val < 0.05 else 'green' # statistical significance = 95% for assuming normality 
    return 'color: %s' % color

In [81]:
def data_count(df, s):  #conteo de filas de tras la agrupación según una variable categórica
    T = (df >>
     group_by(X[s]) >>
     summarize(total = X[s].count())) # da el resultado en porcentaje 
    return (T)

In [80]:
# Para añadir etiquetas en los gráficos

def add_value_labels(ax, spacing=5):
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.1f}".format(y_value)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va)                      # Vertically align label differently for
                                        # positive and negative values.

ANÁLISIS DESCRIPTIVO DE LOS DATOS

El dataset detalla un total de 71 parámetros clínicos para un total de 10.000 pacientes virtuales simulados mediante un modelo poblacional PopPBPK. Entre estos parámetros se encuentran:
- Un conjunto de variables antropométricas: Edad, Altura, Peso, Género e Índice de Masa Corporal (ICM)
- Un conjunto de variables fisiológicas: Tasa de Filtración Glomerular (TFG), estimado de la TFG (eTFG), nivel de creatinina sérica (CrS) y el volumen y el flujo sanguíneo de los siguientes órganos: cerebro, tripa, bazo, páncreas, estómago, liver, riñón, corazón, pulmón, tejido muscular, tejido adiposo, piel, tejido óseo y timo, y los sistemas arterial y venoso.
- Mediciones de los parámetros farmacocinéticos ABC y Cmáx para siete días con un paso temporal de 1 día.

Además, se incluye un identificador de paciente y el tiempo de ejecución de los datos de cada paciente.

El dataset continene 70 variables cuantitativas contínuas y 1 variable categórica (Género) de tipo numérico (0 = Hombre, 1 = Mujer)

In [21]:
clinical_data = data.drop(['p_no', 'time_s'], axis = 1)

In [22]:
clinical_data.shape  # Dimensión del dataset

(10000, 71)

In [23]:
clinical_data.describe() # Medidas de centralidad y dispersión y tamaño de la distribución del dataset

,Age,Height,Weight,Gender,BMI,V_br,V_gut,V_sp,V_pa,V_st,...,AUCday5,AUCday6,AUCday7,Cmaxday1,Cmaxday2,Cmaxday3,Cmaxday4,Cmaxday5,Cmaxday6,Cmaxday7
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,54.565357,1.751734,68.017950,0.494600,22.494714,1.433930,1.321224,0.378899,0.184108,0.169279,...,1541.700378,1542.156533,1542.295512,172.783126,184.297403,186.183141,186.603422,186.711965,186.742595,186.751777
std,17.363696,0.124690,9.390194,0.499996,4.482264,0.105196,0.115395,0.280242,0.053523,0.020091,...,963.267215,964.728643,965.203787,35.279778,45.693222,48.623591,49.457835,49.707347,49.785235,49.810411
min,25.000660,1.279530,45.026925,0.000000,10.946007,1.130813,0.906706,0.000014,0.004382,0.091162,...,44.106872,44.106872,44.106872,27.440705,27.440705,27.440705,27.440705,27.440705,27.440705,27.440705
25%,39.647088,1.665259,60.974996,0.000000,19.265832,1.354615,1.242757,0.149811,0.148451,0.155460,...,870.323592,870.323593,870.323593,149.461141,152.892624,152.985248,153.025065,153.025072,153.025072,153.025072
50%,54.395849,1.752004,67.707499,0.000000,21.910989,1.430995,1.319261,0.324467,0.184301,0.169060,...,1293.325064,1293.325271,1293.325281,174.759033,182.603989,183.163478,183.228519,183.232530,183.232863,183.232891
75%,69.580793,1.836696,74.808722,1.000000,25.228052,1.512143,1.396756,0.553108,0.219054,0.182554,...,1947.167426,1947.170591,1947.170814,198.592344,214.613721,216.265995,216.555337,216.555564,216.562968,216.562970
max,84.996893,2.278747,89.979526,1.000000,44.569439,1.832614,1.804090,2.124067,0.378742,0.242467,...,7143.663028,7186.640709,7203.603255,335.513514,379.590644,421.987752,438.721156,445.325541,447.932177,448.960971


In [24]:
clinical_data.dtypes.value_counts()  # tipos de variables clínicas (excluyendo el identificador de paciente y el tiempo de ejecución)

float64    70
int64       1
dtype: int64

ESTUDIO DE LA RELACIÓN ENTRE LAS VARIABLES INDEPENDIENTES ANTROPOMÉTRICAS Y FISIOLÓGICAS Y EL RESULTADO FARMACOCINÉTICO AL SÉPTIMO DÍA

1. Estudio de normalidad

Selección de variables independientes (variables antropométricas y fisiológicas) categóricas (Género) y cuantitativas, y las variables dependientes de salida ABC y Cmáx de Benazeprilato (metabolito, principio activo) al séptimo día.

In [42]:
data_col = data.columns[np.r_[1:44, 64, 71] ]
data_col

Index(['Age ', 'Height', 'Weight', 'Gender', 'BMI', 'V_br', 'V_gut', 'V_sp',
       'V_pa', 'V_st', 'V_li', 'V_ki', 'V_hr', 'V_lu', 'V_mu', 'V_adi', 'V_sk',
       'V_bo', 'V_th', 'V_art', 'V_vein', 'V_Giu', 'V_Gim', 'V_Gil', 'Q_br',
       'Q_gut', 'Q_sp', 'Q_pa', 'Q_st', 'Q_li', 'Q_ki', 'Q_hr', 'Q_lu', 'Q_mu',
       'Q_adi', 'Q_sk', 'Q_bo', 'Q_th', 'Q_art', 'Q_vein', 'GFR', 'Scr',
       'eGFR', 'AUCday7', 'Cmaxday7'],
      dtype='object')

Se seleccionan ademas 5000 individuos aleatorios para hacer el test de normalidad puesto que para poblaciones de número de individuos superiores a esa cifra los resultados pueden no ser precisos

In [49]:
norm_data = data.sample(n = 5000, replace = False)

-----
Test de normalidad de Shapiro-Wilk

- Hipótesis nula (H0): los datos siguen una distribución normal
- Si el p-value es menor de α = 0.05 entonces se puede descartar la hipótesis nula y por lo tanto la distribución normal de los datos. 

Se estudia la normalidad de las variables independientes y de las variables dependientes de salida ABC y Cmáx mediante el test de Shapiro.

In [55]:
normality_sha = pd.DataFrame(index = data_col, columns = ['p-value'])

result = []
for col in data_col:
    # Shapiro test for each variable
    result += [stats.shapiro(norm_data.loc[:, col])[1]]
normality_sha['p-value'] = result

normality_sha.style.applymap(_color_red_or_green)

/Users/lucia/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.


,p-value
Age,0.000000
Height,0.716845
Weight,0.000000
Gender,0.000000
BMI,0.000000
V_br,0.000000
V_gut,0.000734
V_sp,0.000000
V_pa,0.184620
V_st,0.008792


Para la mayoría de las variables el p-value es menor de 0.05 y se puede descartar la hipótesis nula, por lo que se puede descartar que las variables presenten una ditribución normal. En las únicas variables en las que se puede asumir una distribución normal de los datos son en la variable Altura y en el volumen y el flujo sanguíneo de algunos órganos. Las variables de volumen y flujo sanguíneo del sistema arterial y del sistema venoso pueden ser descartadas del estudio ya no que no presentan variaciones entre los diferntes pacientes, con un rango de 0.

Para en una búsqueda de alcanzar la distribución normal de las variables, se realizan una serie de transformaciones de los datos:
- Transformación logarítmica ln x
- Transformación exponencial  $x^2$
- Transformación radical $\sqrt{x}$
- Transformación inversa  $\frac{1}{x}$

In [60]:
log_cols = list(data_col)
log_cols.remove('Gender')

log_data = np.log(norm_data)    # ln x
sq_data = norm_data**2  # x^2
sqrt_data = np.sqrt(norm_data) # sqrt x
inv_data = 1 / norm_data    # 1 / x


normality_transf_sha = pd.DataFrame(index = data_col, columns = ['ln x', 'sq x', 'sqrt x', '1 / x'])

for i,j in zip([log_data, sq_data, sqrt_data, inv_data], ['ln x', 'sq x', 'sqrt x', '1 / x']):
    result = []
    for col in data_col:
    # Shapiro test for each variable
        result += [stats.shapiro(i.loc[:, col])[1]]
    normality_transf_sha[j] = result

normality_transf_sha.style.applymap(_color_red_or_green)

/Users/lucia/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
/Users/lucia/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.


,ln x,sq x,sqrt x,1 / x
Age,0.000000,0.000000,0.000000,0.000000
Height,0.000002,0.000000,0.109067,0.000000
Weight,0.000000,0.000000,0.000000,0.000000
Gender,1.000000,0.000000,0.000000,1.000000
BMI,0.001523,0.000000,0.000000,0.000000
V_br,0.000000,0.000000,0.000000,0.000000
V_gut,0.002303,0.000000,0.959384,0.000000
V_sp,0.000000,0.000000,0.000000,0.000000
V_pa,0.000000,0.000000,0.000000,0.000000
V_st,0.000000,0.000000,0.114729,0.000000


Ninguna de las transformaciones de los datos impide que se rechaze la hipótesis nula para la mayoría de variables independientes y dependientes. Por lo tanto, en siguientes pasos no podremos asumir la distribución normal de los datos por lo que se hará uso de test estadísticos no parámetricos.

------
2. Estudio de la relación entre las medias de las variables independientes y la disposición de fármaco al séptimo día.

Se desea comparar las medias de las variables independientes (antropométricas y farmacocinéticas) entre los diferentes grupos de disposición de fármaco en el día 7. Para ello, se categorizan las variables farmacocinéticas ABC y Cmax de metabolito al día 7 diferenciando tres categorías en función a la disposición de fármaco: Sub-disposición para valores de ABC < 474 o valores de Cmax < 131, Disposición adecuada cuando los valores de los práemtros se encuentran dentro de los intervalos de confianza estimados de análisis clínicos y sobre-disposición para valores de ABC > 1210 o valores de Cmax > 219.

Debido a que se ha rechazado la condición de normalidad en la mayoría de variables y no se pueden satisfacer las condiciones del test ANOVA, se hace uso del test no paramétrico de Kruskal-Wallis. Entre las condiciones del test de Kruskal-Wallis se encuentran:
- Homocedasticidad: es requisito que todos los grupos tengan la misma varianza
- Misma distribución para todos los grupos, sin necesidad de que esta distribución sean normal

La mayoría de la bibliografía consultada consideran que el ANOVA es bastante robusto a la falta de normalidad sobre todo con muestras medianas o grandes. Solo recomiendan el uso del test de Kruskal-Wallis cuando las poblaciones a comparar sean claramente asimétricas, se cumpla que todas lo sean en la misma dirección y que la varianza sea homogénea. Si la varianza no es homogénea el test adecuado es un ANOVA con corrección de Welch. 

https://www.cienciadedatos.net/documentos/20_kruskal-wallis_test

Categorización de las variables contínuas de farmacocinética al séptimo día:

In [72]:
data['AUC_disp'] = data['AUCday7'].map(AUC_subgroups)
data.AUC_disp = pd.Categorical(data.AUC_disp, 
                      categories=["Sub-disposición","Disposición adecuada","Sobre-disposición"],
                      ordered=True)
data['Cmax_disp'] = data['Cmaxday7'].map(Cmax_subgroups)

data.Cmax_disp = pd.Categorical(data.Cmax_disp, 
                      categories=["Sub-disposición","Disposición adecuada","Sobre-disposición"],
                      ordered=True)

Distribución de la población en función de la variable categórica Distribución de fármaco

In [82]:
x1 = data_count(data, 'AUC_disp'), data_count(data, 'Cmax_disp')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 7), layout='constrained')

axs[0].bar(x1[0].AUC_disp, x1[0].total, color=['lightgrey', 'lightblue', 'lightyellow'], edgecolor = 'black')
axs[0].set_title("Disposición de fármaco dependiente de ABC", fontsize=13)
axs[1].bar(x1[1].Cmax_disp, x1[1].total, color=['lightgrey', 'lightblue', 'lightyellow'], edgecolor = 'black')
axs[1].set_title("Disposición de fármaco dependiente de Cmax", fontsize=13)
plt.suptitle('Distribución de la población según Disposición de fármaco',fontsize=15)
add_value_labels(axs[0])
add_value_labels(axs[1])
fig.supylabel('Número de pacientes')

**Comprobación de Homocedasticidad**

Test de Bartlett para comprobar si los grupos tienen varianzas iguales. Si el p-value es bajo se rechaza la hipótesis nula de que todos los grupos tienen la misma varianza

In [100]:
auc_group = data.groupby('AUC_disp')
sub_auc, adq_auc, over_auc = auc_group.get_group('Sub-disposición'), auc_group.get_group('Disposición adecuada'), auc_group.get_group('Sobre-disposición')

cmax_group = data.groupby('Cmax_disp')
sub_cmax, adq_cmax, over_cmax = cmax_group.get_group('Sub-disposición'), cmax_group.get_group('Disposición adecuada'), cmax_group.get_group('Sobre-disposición')

In [105]:
homocesticity = pd.DataFrame(index = data_col, columns = ['AUC_disp', 'Cmax_disp'])
result1=[]
result2 = []

for col in data_col:
    result1 += [stats.bartlett(sub_auc[col], adq_auc[col], over_auc[col])[1]]
    result2 += [stats.bartlett(sub_cmax[col], adq_cmax[col], over_cmax[col])[1]]

homocesticity['AUC_disp'] = result1
homocesticity['Cmax_disp'] = result2

homocesticity.style.applymap(_color_red_or_green)

,AUC_disp,Cmax_disp
Age,0.448258,0.448430
Height,0.266864,0.504312
Weight,0.000000,0.000000
Gender,0.272599,0.538337
BMI,0.000000,0.000000
V_br,0.749720,0.552657
V_gut,0.879017,0.165358
V_sp,0.883884,0.233082
V_pa,0.820890,0.686062
V_st,0.349139,0.110606


No se cumple la condición de homocedasticidad por lo que es adecuado el test de Kruskal-Wallis. Sin embargo, según la revisión bibliográfica el test ANOVA es robusto cuando se trata de muestras medianas o grandes. Si la varianza no es homogénea, es decir no se cumple la condición de homocedasticidad, el test adecuado es ANOVA con corrección de Welch.

----
**Test de ANOVA**

Las hipótesis contrastadas en un ANOVA de un factor son:

- H0: No hay diferencias entre las medias de los diferentes grupos : μ1=μ2...=μk=μ
- H1: Al menos un par de medias son significativamente distintas la una de la otra.

Condiciones de análsis de ANOVA:
- Distribución normal --> A pesar de que el ANOVA es bastante robusto aun cuando existe cierta falta de normalidad, si la simetría es muy pronunciada y el tamaño de cada grupo no es muy grande, se puede recurrir en su lugar al test no paramétrico prueba H de Kruskal-Wallis
- Homocedasticidad --> Esta condición es más importante cuanto menor es el tamaño de los grupos. El ANOVA es bastante robusto a la falta de homodedasticidad si el diseño es equilibrado. Si no se puede aceptar la homocedasticidad, se recurre a lo que se conoce como ANOVA heterodástica que emplea la corrección de Welch (Welch test)

https://www.cienciadedatos.net/documentos/19_anova

Como no se peude aceptar la homocedasticidad en algunas de las variables estudiadas, además de qeu los grupos no están equilibrados, aunque todos son de tamaños entre mediano y grande, se recurre al test de ANOVA con la corrección de Welch.

---

**Test de ANOVA con la corrección de Welch**

Se estudia la variabilidad de las medias de las variables fisiológicas entre los grupos de Disposición de fármaco definidos en función a los dos parámetros farmacocinéticos. Primero se hace el estudio definiendo los grupos según los niveles de la variable Disposición según ABC y después se realiza el mismo estudio con la variable Disposición según Cmax. Es de esperar que los resultados de ambos estudios sean similares ya que los dos parámetros farmacocinéticos se encuentran correlacionados linealmente (según el análisis de correlación mediante test de Spearman realizado en la siguiente sección)

In [178]:
# Disposición de fármaco según ABC

welch_cols = [elem for elem in list(data_col) if elem not in ['V_art', 'Q_art', 'V_vein', 'Q_vein', 'AUCday7', 'Cmaxday7']]    # se excluyen las variables que se mantienen constantes para todos los pacientes y las variables farmacoci´neticas

welch = {}
res = {}
for i in welch_cols:
        WelchResults = pg.welch_anova(dv=i, between = 'AUC_disp', data = data)['p-unc'][0]
        welch[i] = WelchResults

welch_f = dict(sorted(welch.items(), key=lambda item: item[1]))
for key in welch_f:
    # testing for data type and then condition, order is imp.
    if welch_f[key] < 0.01:
        res[key] = welch_f[key]

welch_f_df = pd.DataFrame(res.items(), columns=['Variable', 'P-value'])
welch_f_df

,Variable,P-value
0,Weight,0.000000e+00
1,BMI,0.000000e+00
2,V_adi,0.000000e+00
3,GFR,0.000000e+00
4,Scr,0.000000e+00
5,eGFR,0.000000e+00
6,V_sk,8.644621e-93
7,Q_pa,1.829894e-66
8,Q_st,1.829894e-66
9,Q_ki,5.815225e-66


In [179]:
# Disposición de fármaco según Cmax

welch_cols = [elem for elem in list(data_col) if elem not in ['V_art', 'Q_art', 'V_vein', 'Q_vein', 'AUCday7', 'Cmaxday7']]    # se excluyen las variables que se mantienen constantes para todos los pacientes y las variables farmacoci´neticas

welch = {}
res = {}
for i in welch_cols:
        WelchResults = pg.welch_anova(dv=i, between = 'Cmax_disp', data = data)['p-unc'][0]
        welch[i] = WelchResults

welch_f = dict(sorted(welch.items(), key=lambda item: item[1]))
for key in welch_f:
    # testing for data type and then condition, order is imp.
    if welch_f[key] < 0.01:
        res[key] = welch_f[key]

welch_f_df = pd.DataFrame(res.items(), columns=['Variable', 'P-value'])
welch_f_df

,Variable,P-value
0,Weight,0.000000e+00
1,BMI,0.000000e+00
2,V_adi,0.000000e+00
3,Scr,0.000000e+00
4,eGFR,1.138688e-318
5,GFR,4.110486e-187
6,V_sk,9.432703e-117
7,Q_ki,7.889375e-82
8,Q_pa,9.351305e-82
9,Q_st,9.351305e-82


**CONCLUSIONES** del test de ANOVA con corrección de Welch: para esta lista de variables el p-value < 0.01 y se puede rechazar la hipótesis nula de que la media de los parámetros entre los grupos de disposición de fármaco son iguales. Estas variables difieren significativamente entre los diferentes grupos de disposición farmacológica. Es notable además que la lista de variables significativas es muy similar cuando la disposición farmacológica viene definida tanto por la variable ABC como por la variable Cmáx. Esto tiene coherencia con la hipótesis de que ambos parámetros estan relacionados de forma lineal. Para el estudio de la correlación lineal de los parámetros farmacocinéticos se implementa el test de correlación de Spearman en la siguiente sección.

-----


**Estudio de correlación entre las variables farmacocinéticas ABC y Cmax t = 7. Test no paramétrico de Spearman**

Como se ha comprobado anteriormente, no se puede asumir la condición de normalidad para las variables farmacocinéticas ABC y Cmax, por lo que para el análisis de correlación se debe hacer uso de un test no paramétrico que no asuma la normalidad de los datos. Es por ello que se realiza un test de correlación de Spearman.

Los coeficientes de correlación lineal son estadísticos que cuantifican la asociación lineal entre dos variables numéricas: Rho de Spearman. Su valor está comprendido en el rango [+1 , -1]. Siendo +1 una correlación positiva perfecta y -1 una correlación negativa perfecta. Un valor de 0 indica asociacion nula entre las variables numéricas.

https://www.cienciadedatos.net/documentos/pystats05-correlacion-lineal-python.html


* Los gráficos de distribución de las mediciones de las variables ABC y Cmáx en el séptimo día se han generado en un archivo .Rmd (R) como material suplementario de este Trabajo de Fin de Grado que puede encontrarse en: *Material-complementario/Gráficos_adicionales.Rmd*

In [ ]:
spearman_data = data[['p_no', 'AUCday7', 'Cmaxday7']]

In [229]:
# Estimación de correlación y significancia con la librería Scipy
r, p = stats.spearmanr(spearman_data['AUCday7'], spearman_data['Cmaxday7'])
print(f"Correlación Spearman: r={r}, p-value={p}")

Correlación Spearman: r=0.9182374049303742, p-value=0.0


In [230]:
# Estimación de correlación, significancia e intervalos de confianza con la librería Pingouin

display(pg.corr(spearman_data['AUCday7'], spearman_data['Cmaxday7'], method='spearman'))

,n,r,CI95%,p-val,power
spearman,10000,0.918237,"[0.92, 0.92]",0.0,1.0


**CONCLUSIONES:** Los test estadísticos muestran una correlación lineal positiva muy alta, con evidencias estadísticas (p-value = 0) de que la relación observada no es producto del azar

* La gráfica de distribución de la variable ABC respecto de Cmáx, junto con la representación de linealidad, se han generado en el archivo de material suplementario (ver: *Material-complementario/Gráficos_adicionales.Rmd*)

---

3. Análisis de regresión lineal múltiple

La regresión lineal múltiple permite generar un modelo lineal en el que el valor de la variable dependiente o respuesta (Y) se determina a partir de un conjunto de variables independientes llamadas predictores (X1, X2, X3...). Los modelos de regresión múltiple pueden emplearse para predecir el valor de la variable dependiente o para evaluar la influencia que tienen los predictores sobre ella.

En este proyecto se realiza un modelado lineal mediante regresión lineal múltiple que permita evaluar la influencia que tienen los predictores clínicos sobre el resultado farmacocinético con la intención de realizar una selección de predictores que permitan la estratificación poblacional y la definición de los estados de un modelo computacional de aprendizaje por refuerzo.

Ecuaión de modelos lineales múltiples:

Yi=(β0+β1X1i+β2X2i+⋯+βnXni)+ei

Es importante tener en cuenta que la magnitud de cada coeficiente parcial de regresión depende de las unidades en las que se mida la variable predictora a la que corresponde, por lo que su magnitud no está asociada con la importancia de cada predictor. Para poder determinar qué impacto tienen en el modelo cada una de las variables, se emplean los coeficientes parciales estandarizados, que se obtienen al estandarizar (sustraer la media y dividir entre la desviación estándar) las variables predictoras previo ajuste del modelo.

**Condiciones para la RLM**

- No colinealidad o multicolinialidad: los predictores deben ser independientes. Hay colinealidad cuando uno o varios predictores están linealmente relacionados o se puede expresar como combinación lineal de otros predictores. Si hay colinealidad, no se peude estudiar el efecto individual de cada variable colineal de forma precisa. Se puede determinar la existencia de colinialidad entre los predictores del modelo de rgresión mediante una matriz de correlación, que estudia la relación lineal de cada par de predictores. Si se detecta colinealidad, se puede excluir uno de los predictores colineales de forma consciente, conservando el que según el investigador está influyendo realmente en la variable respuesta.
- Parsimonia: el mejor modelo es el capaz de explicar con mayor precision la variabilidad observada en la variable respuesta con el menor numero de predictores
- Linealidad entre los predictores numéricos y la variable respuesta -> análisis de residuos (o test de Spearman no?)
Entre otros

El término "lineal" en los modelos de regresión hace referencia al hecho de que los parámetros se incorporan en la ecuación de forma lineal, no a que necesariamente la relación entre cada predictor y la variable respuesta tenga que seguir un patrón lineal.

(LUCIA: esto no lo pongo en el TFG, solo lo que he hecho, la matriz de correlación para ver la colinealidad y si hago lo de ver la linealidad y comentar la parsimonia pero no digas más)

**Selección de predictores**

La regresión lineal múltiple es un método estadístico que trata de modelar la relación entre una variable continua y dos o más variables independientes mediante el ajuste de una ecuación lineal. Es importante hacer una selección de predictores que aporten información relevante. 

Estrategias para la selección de predictores:
- Subset selection: proceso iterativo que vaya descartando los predictores menos relevantes.
- Regularización Ridge, Lasso o Elastic Net: fuerzan que los coeficientes del modelo tiendan a cero
- Reducción de dimensionalidad

En este trabajo se realiza una selección de predictores del modelo lineal siguiendo la estrategia de subset selection. Los métodos conocidos como subset selection tienen la finalidad de identificar y seleccionar, de entre todos los predictores disponibles, aquellos que están más relacionados con la variable respuesta y así crear el mejor modelo. 

Esquema general:
- Crear un conjunto de modelos candidatos (todos los posibles o un conjunto considerable de ellos), mediante diferentes combinaciones de los predictores disponibles.
- Para cada posible tamaño de modelo (1 predictor, 2 predictores…) se selecciona el mejor basándose en el error de entrenamientor.
- Los modelos finalistas de cada tamaño se comparan entre ellos para una métrica de validación (validación cruzada, Cp, AIC, BIC o R2ajustado) y se identificar el mejor.

Dentro de los métodos de subset selection hay diversas posibilidad. Debido a las limitaciones computacionales de Best subset selection, que lo hacen inviable para más de 40 predictores, y probelmas de posible overfitting, se lleva implementa un método de Stepwise Selection: Backward Stepwise selection. El concepto es equivalente al de forward stepwise selection pero, en este caso, iniciando el proceso a partir del modelo que contiene todos los posibles predictores (full model Mk). En cada iteración, se entrenan todos los modelos que se pueden crear eliminando un único predictor y se selecciona el que tiene menor error de entrenamiento tiene. El proceso se repite hasta llegar al modelo nulo sin predictores (M0).

También se prueba con modelos de regularización (shirnkage) --> los explico en otro momento que voy a ver si me sale o no me sale
- Ridge
- LAsso
- Elastic Net
- Lambda Seach

El método paso a paso requiere de algún criterio matemático para determinar si el modelo mejora o empeora con cada incorporación o extracción. Existen varios parámetros empelados, de entre los que destacan el Cp, AIC, BICy R2ajustado, cada uno de ellos con ventajas e inconvenientes. El método Akaike(AIC) tiende a ser más restrictivo e introducir menos predictores que el R2-ajustado. Para un mismo set de datos, no todos los métodos tienen porque concluir en un mismo modelo.

Es frecuente encontrar ejemplos en los que la selección de predictores se basa en el p-value asociado a cada uno. Si bien este método es sencillo e intuitivo, presenta múltiples problemas: la inflación del error tipo I debida a las comparaciones múltiples, la eliminación de los predictores menos significativos tiende a incrementar la significancia de los otros predictores … Por esta razón, a excepción de casos muy sencillos con pocos predictores, es preferible no emplear los p-values como criterio de selección

**Stepwise (paso a paso) dirección backward

**Validación y test**

Una vez seleccionado el mejor modelo que se puede crear con los datos disponibles, se tiene que comprobar su capacidad prediciendo nuevas observaciones que no se hayan empleado para entrenarlo, de este modo se verifica si el modelo se puede generalizar. Una estrategia comúnmente empleada es dividir aleatoriamente los datos en dos grupos (70%-30%), ajustar el modelo con el primer grupo y estimar la precisión de las predicciones con el segundo. Para una descripción más detallada de las estrategias de validación consultar: Validación de modelos de regresión: Cross-validation, OneLeaveOut, Bootstrap y Machine Learning con R y caret.


Fuentes:
- https://www.cienciadedatos.net/documentos/25_regresion_lineal_multiple.html
- https://rpubs.com/Joaquin_AR/242707


**Análisis de relacion entre variables**

Esta información es crítica a la hora de identificar cuáles pueden ser los mejores predictores para el modelo, qué variables presentan relaciones de tipo no lineal (por lo que no pueden ser incluidas) y para identificar colinialidad entre predictores. Estudio de colinealidad: correlación entre pares de variables numéricas. Matriz de correlación

In [242]:
#selección variables numéricas
data.select_dtypes(include=['float64', 'int'])

corr_matrix = data[data_col].corr(method='spearman')  #matriz de correlación

def tidy_corr_matrix(corr_mat):
    '''
    Función para convertir una matriz de correlación de pandas en formato tidy.
    '''
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    
    return(corr_mat)

tidy_corr_matrix(corr_matrix).head(10)


,variable_1,variable_2,r,abs_r
1091,Q_st,Q_pa,1.000000,1.000000
788,V_Giu,V_st,1.000000,1.000000
1427,Q_bo,Q_sk,1.000000,1.000000
388,V_st,V_Giu,1.000000,1.000000
1051,Q_pa,Q_st,1.000000,1.000000
1466,Q_th,Q_mu,1.000000,1.000000
1387,Q_sk,Q_bo,1.000000,1.000000
1306,Q_mu,Q_th,1.000000,1.000000
1129,Q_li,Q_br,0.999945,0.999945
929,Q_br,Q_li,0.999945,0.999945


Muchas de las variables están altamente correlacionadas (correlación absoluta > 0.8), lo que supone un problema a la hora de emplear modelos de regresión lineal.

**Ajuste del modelo**

In [277]:
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

Se ajustan varios modelos lineales con y sin regularización, con el objetivo de identificar cuál de ellos es capaz de predecir mejor. Para poder evaluar la capacidad predictiva de cada modelo, se dividen las observaciones disponibles en dos grupos: uno de entrenamiento (70%) y otro de test (30%).

In [364]:
X = data[data_col[0:43]]
X = X.drop(['Q_art', 'Q_vein', 'V_art', 'V_vein'], axis = 1)
y = data[data_col[-2]]    #  La variable resultado de este análisis es la variable ACB al séptimo día 

In [300]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [301]:
# Creación del modelo utilizando matrices como en scikitlearn---> TODOS LOS PREDICTORES
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.OLS(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     5659.
Date:                Wed, 17 May 2023   Prob (F-statistic):               0.00
Time:                        23:53:28   Log-Likelihood:                -54786.
No. Observations:                8000   AIC:                         1.096e+05
Df Residuals:                    7975   BIC:                         1.098e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.704e+04   1541.523    -11.055      0.0

El modelo con todas las variables introducidas como predictores tiene un  𝑅2 alto (0.945), es capaz de explicar el 94.5% de la variabilidad observada en las ventas. P.value significativo por lo que se acepta que el modelo es mejor que lo esperado por azar. Al menos uno de los coefiencientes es menor que 1. Stepwise backwards vamos quitando parametros

Se eliminaron las variables con p-values no significativos

In [367]:
X = X.drop(['Weight', 'V_br', 'V_gut', 'V_sp', 'V_pa', 'V_li', 'V_hr', 'V_lu', 'V_mu', 'V_adi', 'V_sk', 'V_bo', 'V_th', 'V_Gim', 'V_Gil', 'Q_pa', 'Q_st', 'Q_mu'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

# Creación del modelo utilizando matrices como en scikitlearn---> TODOS LOS PREDICTORES
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.OLS(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                 1.055e+04
Date:                Thu, 18 May 2023   Prob (F-statistic):               0.00
Time:                        02:01:01   Log-Likelihood:                -55386.
No. Observations:                8000   AIC:                         1.108e+05
Df Residuals:                    7988   BIC:                         1.109e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.154e+04   1160.932    -44.398      0.0

Una primera selección de parámetros manual o "consciente" en función a los resultados de los modelos de regresión previos y al conocimiento sobre el simulador poblacional del que proceden los datos. Se prueba a seleccionar como predictores únicamente las variables que sirven como entrada del modelo computacional: variables antropométricas (excepto Peso, que fue descartada en el paso anterior debido a su baja significancia) y los parámetros que determinan la condición renal: CrS y TFG

In [360]:
X = data[['Gender', 'BMI', 'Scr', 'GFR', 'Age ', 'Height']]
y = data[data_col[-2]] 

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [358]:
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.OLS(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.907
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                 1.560e+04
Date:                Thu, 18 May 2023   Prob (F-statistic):               0.00
Time:                        00:11:21   Log-Likelihood:                -56852.
No. Observations:                8000   AIC:                         1.137e+05
Df Residuals:                    7994   BIC:                         1.138e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9368.1984     83.548   -112.129      0.0

Se realiza una selección de predictores basada en el conocimiento del dominio del problema global del proyecto y se ajusta el modelo lineal únicamente con esos predictores. Para la selección consciente de predictores se tienen en cuenta las siguientes consideraciones:
<br>
- Los resultados del análisis de variabilidad entre las medias de los parámetros antropométricos y fisiológicos de los pacientes de los distintos grupos de disposición de fármaco. Listas de variables con diferencias significativas entre sus medias, lo que indica que los datos provienen de poblaciones diferentes.
<br>
- Los resultados del análisis de regresión lineal
<br>
- La naturaleza de los datos sintéticos generados por el modelo poblacional que recibe como variables input los parámetros antropométricos y GFR, a partir de los cuales recalcula el resto de variables fisiológicas

- Las limitaciones prácticas de accesibilidad a los datos de ciertas variables, como el tamaño o flujo sanguíneo de los órganos internos, en una futura implementación del modelo computacional en datos clínicos de pacientes reales.

- Las limitaciones bibliográficas sobre intervalos o rangos correctamente definidos de algunas variables que permitan hacer una estratificación poblacional adecuada.  <font color='red'>Además de las referencias de no usar Scr como parámetro para evaluar el nivel de insuficiencia renal de un paciente</font> 

Se seleccionaron 3 parámetros que definen los grupos de estratificación además los estados del modelo computacional, junto a los valores de medición de las variables ABC y Cmáx

- ICM: índice de masa corporal. Se selecciona por ser una variable correlacionada con la Altura y el Peso ya que es calculada a partir de la combinación lineal de las otras dos, por lo tiene influencia de las dos variables que son variables que aparecen significativas en los estudios de variabilidad
- TFG: Tasa de Filtración Glomerular
- Género. 

 <font color='red'> Explicar mejor por que se elige cada una</font> 
